In [ ]:
from typing import Optional

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
def add_s_layers(river_ds):

    # Creating new array to fill the gaps
    v_data = np.zeros((5, 175))
    v_add = xr.DataArray(v_data, dims=river_ds['river_Vshape'].dims, coords=river_ds['river_Vshape'].coords)
    temp_data = np.ones((4019, 5, 175)) * 5
    t_add = xr.DataArray(temp_data, dims=river_ds['river_temp'].dims, coords=river_ds['river_temp'].coords)
    salt_data = np.ones((4019, 5, 175))
    s_add = xr.DataArray(salt_data, dims=river_ds['river_salt'].dims, coords=river_ds['river_salt'].coords)
    
    # New ready data arrays
    river_Vshape = xr.concat([v_add, river_ds['river_Vshape']], dim="s_rho")
    river_temp = xr.concat([t_add, river_ds['river_temp']], dim="s_rho")
    river_salt = xr.concat([s_add, river_ds['river_salt']], dim="s_rho")
    
    # Remove old and add new variables with new s_rho=40 dimension
    river_ds = river_ds.drop_dims('s_rho')
    river_ds = river_ds.assign(river_Vshape=river_Vshape, river_temp=river_temp, river_salt=river_salt)

    return river_ds

In [ ]:
def mask_all_rivers(ds):
    ds['river_transport'].values = np.zeros((4019, 175))
    return ds

In [ ]:
def mask_some_rivers(ds, rivers):
    """
    Assignment uses i locations (implicit)
    """

    ds.river_transport[dict(river=rivers)] = np.zeros((4019, 1))
    return ds

In [ ]:
def check_rivers_runoff(ds, verbose=False):
    river_xi_eta_dir = []
    for i in range(ds.dims['river']):
        river = ds.isel(river=i, river_time=0)
        assert i+1 == int(river.river)
        cell_runoff = (abs(river.river_transport) * river.river_Vshape).max().values
        if cell_runoff > 10:
            if verbose:
                print(f"River {i+1} max runoff: {cell_runoff}")
                print(f"Coordinates: {river.river_Xposition}; {river.river_Eposition}")
                print(f"Direction: {river.river_direction} \n")
            river_xi_eta_dir.append((
                int(river.river),
                int(river.river_Xposition.values), 
                int(river.river_Eposition.values),
                int(river.river_direction.values)
            ))
    return river_xi_eta_dir

In [ ]:
def save_rivers(ds, name):
    ds.to_netcdf(path=f'fram_data/norfjords_160m_river_{name}.nc', format='NETCDF4')

Check rivers

In [ ]:
ds = xr.open_dataset('fram_data/norfjords_160m_river_v2.nc')
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_version2+.nc')
river_xi_eta_dir = check_rivers_runoff(ds, verbose=False)
rivers = [river - 1 for river, _, _, _ in river_xi_eta_dir]  # assignment use iloc

In [ ]:
def plot_river_estuary_u(river, river_xi, river_eta, river_dir):

    assert river_dir == 0
    mask_u = grid_ds.mask_u.copy()
    plt.figure(figsize=(15, 4))

    plt.subplot(121)
    mask_u.isel(
        xi_u=slice(river_xi - 4, river_xi + 5), 
        eta_u=slice(river_eta - 4, river_eta + 5)
        ).plot()

    mask_u[dict(xi_u=river_xi, eta_u=river_eta)] = np.nan

    plt.subplot(122)
    mask_u.isel(
        xi_u=slice(river_xi - 4, river_xi + 5), 
        eta_u=slice(river_eta - 4, river_eta + 5)
        ).plot()
    # to show the fortran locations
    plt.suptitle(f"River: {river}; Xi: {river_xi+1}; Eta: {river_eta}; Direction: {river_dir}.")
    # plt.suptitle(f"River: {river}; Xi: {river_xi+1}; Eta: {river_eta+1}; Direction: {river_dir}.")


def plot_river_estuary_v(river, river_xi, river_eta, river_dir):

    assert river_dir == 1
    mask_v = grid_ds.mask_v.copy()
    plt.figure(figsize=(15, 4))

    plt.subplot(121)
    mask_v.isel(
        xi_v=slice(river_xi - 4, river_xi + 5), 
        eta_v=slice(river_eta - 4, river_eta + 5)
        ).plot()

    mask_v[dict(xi_v=river_xi, eta_v=river_eta)] = np.nan

    plt.subplot(122)
    mask_v.isel(
        xi_v=slice(river_xi - 4, river_xi + 5), 
        eta_v=slice(river_eta - 4, river_eta + 5)
        ).plot()
    # to show the fortran locations
    plt.suptitle(f"River: {river}; Xi: {river_xi}; Eta: {river_eta+1}; Direction: {river_dir}.")
    # plt.suptitle(f"River: {river}; Xi: {river_xi+1}; Eta: {river_eta+1}; Direction: {river_dir}.")


def plot_river_estuary_rho(river, river_xi, river_eta, river_dir):

    mask_rho = grid_ds.mask_rho.copy()
    plt.figure(figsize=(15, 4))

    plt.subplot(121)
    mask_rho.isel(
        xi_rho=slice(river_xi - 4, river_xi + 5), 
        eta_rho=slice(river_eta - 4, river_eta + 5)
        ).plot()

    mask_rho[dict(xi_rho=river_xi, eta_rho=river_eta)] = np.nan

    plt.subplot(122)
    mask_rho.isel(
        xi_rho=slice(river_xi - 4, river_xi + 5), 
        eta_rho=slice(river_eta - 4, river_eta + 5)
        ).plot()
    # to show the fortran locations
    plt.suptitle(f"River: {river}; Xi: {river_xi+1}; Eta: {river_eta+1}; Direction: {river_dir}.")

In [ ]:
def plot_rivers_luv(river_xi_eta_dir):
    """
    Probably, not sure: Due to roms notation where rho points start from zero 
    (so not such points in fortran), eta coordinate in mask_u already "has"
    pythonic coordinates : bullshit - smth probably bad with a rivers forcing.
    https://www.myroms.org/wiki/Grid_Generation

    Args:
        river_xi_eta_dir: a list of river, xi, eta, dir
    """
    for river, xi, eta, direction in river_xi_eta_dir:
        if direction == 0: 
            xi, eta = xi - 1, eta
            plot_river_estuary_u(river, xi, eta, direction)
        else:
            xi, eta = xi, eta - 1
            plot_river_estuary_v(river, xi, eta, direction)


def plot_rivers_lw(river_xi_eta_dir):
    for river, xi, eta, direction in river_xi_eta_dir:
        plot_river_estuary_rho(river, xi-1, eta-1, direction)

Fix rivers

In [ ]:
def split_river(
    ds, 
    number: float, 
    dir_swap: bool = False, 
    flux_swap: bool = False,
    f_xi: Optional[float] = None,
    f_eta: Optional[float] = None,
    s_xi: Optional[float] = None,
    s_eta: Optional[float] = None,
    ):
    """
    Decrease flux twice at the current position.
    Adds a new river at the provided position.

    Args:
        ds: Entire dataset to change.
        number: river coordinate number
        dir_swap: river ROMS direction
        flux_swap: multiplies by -1 if True
        f_xi: current river new xi coordinate
        f_eta: current river eta new coordinate
        s_xi: new river xi coordinate
        s_eta: new river eta coordinate

    Returns:
        ds: a new dataset with an additional 'river'.
    """
    directions = {
        0: 1,
        1: 0,
    }

    river_ds = ds.sel(river=number)

    if dir_swap: 
        direction = river_ds.river_direction.values
        river_ds.river_direction.values = directions[int(direction)]

    if_swap = -1 if flux_swap else 1
    river_ds.river_transport.values = if_swap * 1/2 * river_ds.river_transport.values

    if f_xi is not None: 
        river_ds.river_Xposition.values = f_xi
    if f_eta is not None: 
        river_ds.river_Eposition.values = f_eta

    ds.loc[dict(river=number)] = river_ds

    if s_xi is not None: 
        river_ds.river_Xposition.values = s_xi
    if s_eta is not None: 
        river_ds.river_Eposition.values = s_eta

    river_ds.river.values = ds.river[-1].values + 1
    ds = xr.concat([ds, river_ds], dim="river")

    return ds

River 1

In [ ]:
plot_rivers_luv(river_xi_eta_dir[0:1])

In [ ]:
ds = split_river(ds, number=12., dir_swap=True, flux_swap=True, f_xi=728, f_eta=10, s_xi=728, s_eta=9)

In [ ]:
rivers = ds.sel(river=12), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

River 2

In [ ]:
plot_rivers_luv(river_xi_eta_dir[1:2])

In [ ]:
ds = split_river(ds, number=53., dir_swap=False, flux_swap=False, f_xi=805, f_eta=192, s_xi=805, s_eta=193)

In [ ]:
rivers = ds.sel(river=53), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

River 3

In [ ]:
plot_rivers_luv(river_xi_eta_dir[2:3])

In [ ]:
ds = split_river(ds, number=86., dir_swap=False, flux_swap=False, f_xi=1053, f_eta=96, s_xi=1054, s_eta=96)

In [ ]:
rivers = ds.sel(river=86), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

River 4

In [ ]:
plot_rivers_luv(river_xi_eta_dir[3:4])

In [ ]:
ds = split_river(ds, number=93., dir_swap=False, flux_swap=False, f_xi=1352, f_eta=48, s_xi=1351, s_eta=48)

In [ ]:
rivers = ds.sel(river=93), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

River 5

In [ ]:
plot_rivers_luv(river_xi_eta_dir[4:5])

In [ ]:
ds = split_river(ds, number=150., dir_swap=False, flux_swap=False, f_xi=1284, f_eta=444, s_xi=1285, s_eta=444)

In [ ]:
rivers = ds.sel(river=150), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

River 6

In [ ]:
plot_rivers_luv(river_xi_eta_dir[5:6])

In [ ]:
ds = split_river(ds, number=156., dir_swap=False, flux_swap=False, f_xi=1373, f_eta=541, s_xi=1372, s_eta=541)

In [ ]:
rivers = ds.sel(river=156), ds.isel(river=-1)
new_river_xi_eta_dir = []
for river in rivers:
    new_river_xi_eta_dir.append((
        int(river.river),
        int(river.river_Xposition.values), 
        int(river.river_Eposition.values),
        int(river.river_direction.values)
    ))

In [ ]:
plot_rivers_luv(new_river_xi_eta_dir)

In [ ]:
save_rivers(ds, "v2+")